### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import re
from my_toolkit import key_check
from urllib.parse import quote_plus


In [2]:
# Set environment variables from the .env in the local environment    
my_env_path= 'C:\src\\ai\data-sourcing-challenge\.data-sourcing_challenge.env'
'''
def check_keys(key_path=None):
    try:
        load_dotenv(key_path,override=True)   
        nyt_api_key = os.getenv("NYT_API_KEY")
        tmdb_api_key = os.getenv("TMDB_API_KEY")
        assert nyt_api_key is not None, 'NYT_API_KEY not found in .env file'
        assert tmdb_api_key is not None, 'TMDB_API_KEY not found in .env file'
        responce=requests.get(f'https://api.nytimes.com/svc/mostpopular/v2/viewed/1.json?api-key={nyt_api_key}')
        assert responce.status_code == 200, f'The key provided failed to authenticate nyt_api_key {nyt_api_key} code {responce.status_code}'
        responce=requests.get(f'https://api.themoviedb.org/3/movie/11?api_key={tmdb_api_key}')
        assert responce.status_code == 200, f'The key provided failed to authenticate tmdb_api_key {tmdb_api_key} code {responce.status_code}'
    except Exception as e:
        # Handle potential errors in loading .env or missing API keys
        print(f'An error occurred: {e}')
    else:
        print('All keys laoded correctly')
'''
if key_check(my_env_path):
    nyt_api_key = os.getenv("NYT_API_KEY")
    tmdb_api_key = os.getenv("TMDB_API_KEY")
else:
    print ('fix Keys and rerun')

All keys loaded correctly


### Access the New York Times API

In [3]:
# Sample from web page
# https://api.nytimes.com/svc/search/v2/articlesearch.json?q=new+york+times&page=2&sort=oldest&api-key=your-api-key 

#  Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = quote_plus('section_name:"Movies" AND type_of_material:"Review" AND headline:"love"')

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = quote_plus("headline,web_url,snippet,source,keywords,pub_date,byline,word_count")

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url=(f'{url}fq={filter_query}&begin_date={begin_date}&end_date={end_date}&fl={field_list}&sort={sort}')
display (query_url)

'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name%3A%22Movies%22+AND+type_of_material%3A%22Review%22+AND+headline%3A%22love%22&begin_date=20130101&end_date=20230531&fl=headline%2Cweb_url%2Csnippet%2Csource%2Ckeywords%2Cpub_date%2Cbyline%2Cword_count&sort=newest'

In [4]:
# Create an empty list to store the reviews
love_movie_review=[]
# loop through pages 0-19
for page in range(0,20):
    # Construct the query URL for the current page
    page_url = f'{query_url}&page={page}&api-key={nyt_api_key}'

    try:
        # Attempt to make a "GET" request and parse the JSON response
        response = requests.get(page_url).json()

        # Check if the "docs" list is empty; if so, print a message and exit the loop
        if not (response["response"]["docs"]):
                raise ValueError (f'No results on page {page}, stopping.')
        else: 
            print(f'Checked page  {page}')            
        # Otherwise, process each article in "docs"
        for article in response["response"]["docs"]:
            if article :
                love_movie_review.append(article)
            # diag print(f'article title {article["headline"]["main"]}')
        # Add a twelve second pause between requests to adhere to API query limits
        time.sleep(12)
    except ValueError as e:
        # Handle the case where no documents are found
        print(e)
        break
    except Exception as e:
        # Handle potential errors in the request or data processing
        print(f'An error occurred: {e}')
        break

Checked page  0
Checked page  1
Checked page  2
Checked page  3
Checked page  4
Checked page  5
Checked page  6
Checked page  7
Checked page  8


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print (json.dumps(love_movie_review[:5], indent=4))

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
love_movie_review_df=pd.json_normalize(love_movie_review)
love_movie_review_df.iloc[0:4]

In [ ]:
# Extract the title from the "headline.main" column and
# 
# Regular expression to match text enclosed by ‘ and ’
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019.
# In testing I found that some reviews did not have titles.
# -----End string should include " Review" to avoid cutting title early----
# "" Review doesnt work "Review": ‘What’s Love Got to Do With It?’  not all titles come first. opted for u0020 before and after. 
pattern = r"(?:\u0020|^)\u2018(.+?)\u2019(?:\u003A|\u0020|$)"

def extract_title(headline):
    title_match = re.search(pattern, headline)
    if title_match:
        return title_match.group(1)
    else:
        print (headline)
        return "not found"

# Apply this function to the 'headline.main' column and assign the result to the 'title' column
love_movie_review_df['title'] = love_movie_review_df['headline.main'].apply(extract_title)	
love_movie_review_df[['snippet','title']].head(3)


In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019.
# -----End string should include " Review" to avoid cutting title early----
# 
# In testing I found that some reviews did not have titles.
# I found that some responses did not have title. The revised code prints a message with the headline  page and a message saying title not found. 
# "" Review doesnt work "Review": ‘What’s Love Got to Do With It?’  not all titles come first. 
# opted for u0020 before and after. 

pattern = r"(?:\u0020|^)\u2018(.+?)\u2019(?:\u003A|\u0020|$)"

def extract_title(headline, idx):
    title_match = re.search(pattern, headline)
    if title_match:
        # print(idx,headline)
        return title_match.group(1)
    else:
        print (f'not found {idx, headline}')
    #    return "not found"

# Apply this function to the 'headline.main' column and assign the result to the 'title' column
# Direct iteration over the column to access both the index and value
love_movie_review_df['title'] = love_movie_review_df.apply(lambda row: extract_title(row['headline.main'], row.name), axis=1)
display (love_movie_review_df[['snippet','title']].head(3))

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    extracted_keywords = ";".join(f"{item['name']}: {item['value']}" for item in keyword_list)
    # extracted_keywords = extracted_keywords.rstrip(';')
    return extracted_keywords
# this copy make it possible for me to rerun this without having to start from scratch
lmr_keyword_df = love_movie_review_df.copy(deep=True)

# Fix the "keywords" column by converting cells from a list to a string
lmr_keyword_df['keywords'] = lmr_keyword_df['keywords'].apply(extract_keywords)
# display(lmr_keyword_df[['title','keywords']].head(3))
#
#play time with style
styled_subset_df=lmr_keyword_df.loc[:4,['title','keywords']]
styled_df = styled_subset_df.style.set_table_styles({
    'title': [{'selector': '',
                'props': [('width', '200px'), ('text-align', 'right')]}],
    'keywords': [{'selector': '',
                'props': [('width', '700px'), ('text-align', 'left')]}]
}, overwrite=False).hide(axis=0)
# display(styled_df)
display (lmr_keyword_df)
display (styled_df)
del styled_df

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = love_movie_review_df['title'].to_list()
print("Top 5 Titles:\n" + json.dumps(titles_list[:5], indent= 4, ensure_ascii=False)[1:-1])

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url_query   = "https://api.themoviedb.org/3/search/movie?query="
url_detail  = "https://api.themoviedb.org/3/movie/"

In [ ]:
# Create an empty list to store the results
tmbd_movies_list=[]

# Create a request counter to sleep the requests after a multiple
# of 50 requests
req_counter = 0
'''
https://developer.themoviedb.org/docs/rate-limiting
While our legacy rate limits have been disabled for some time, 
we do still have some upper limits to help mitigate needlessly high bulk scraping. 
They sit somewhere in the 50 requests per second range. 
This limit could change at any time so be respectful of the service we have built and respect the 429 if you receive one.
'''
# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if req_counter % 50 == 0:
        print ('naptime')
        time.sleep(1)
        req_counter=0
    else:
    # Add 1 to the request counter
        req_counter += 1
    # Perform a "GET" request for The Movie Database
    response=requests.get(f'{url_query}{title}"&api_key={tmdb_api_key}')

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id    
        if response.status_code == 200:
        # Get movie id
            data = response.json()
            if data['results']:
                movie_id = data['results'][0]['id']
                # print(f"Movie ID Found: {movie_id}  {req_counter}  {title}")
        # Make a request for the full movie details
        # Execute "GET" request with url
                detail_response = requests.get(f"{url_detail}{movie_id}?api_key={tmdb_api_key}")
                if detail_response.status_code == 200:
                    movie_detail_df = detail_response.json()
                    # display (detail_response.json())
                    # print (json.dumps(movie_detail_df,indent=4))

        # Extract the genre names into a list
                    genres_list = [genre['name'] for genre in movie_detail_df['genres']]
                    # print (genres_list)
        # Extract the spoken_languages' English name into a list
                    spoken_languages = [spoken_language['english_name'] for spoken_language in movie_detail_df['spoken_languages']]
                    # print(spoken_languages)

        # Sample 'spoken_languages': [{'english_name': 'Spanish', 'iso_639_1': 'es', 'name': 'Español'}],
        # Extract the production_countries' name into a list
                    production_countries_name = [production_countries_name['name'] for production_countries_name in movie_detail_df['production_countries']]
                    
                    # 'production_countries': [{'iso_3166_1': 'AR', 'name': 'Argentina'}],
                    # print(production_countries_name)
        # Add the relevant data to a dictionary and
        
        # append it to the tmdb_movies_list list
                    tmbd_movies_list.append({
                    'movie_id'        : movie_id,
                    'movie_title'     : title,
                    'original_title'  : (movie_detail_df['original_title']),
                    'budget'          : (movie_detail_df['budget']),
                    'gendre_list'     : genres_list,
                    'original_language'        : (movie_detail_df['original_language']),
                    'spoken_languages': spoken_languages,
                    'homepage'        : (movie_detail_df['homepage']),
                    'overview'        : (movie_detail_df['overview']),
                    'popularity'      : (movie_detail_df['popularity']),
                    'runtime'         : (movie_detail_df['runtime']),
                    'revenue'         : (movie_detail_df['revenue']),
                    'release_date'    : (movie_detail_df['release_date']),
                    'vote_average'    : (movie_detail_df['vote_average']),
                    'vote_count'      : (movie_detail_df['vote_count']),
                    'production_counties_name': production_countries_name})
                    
                else:
                    raise ("put a exception here") 
        # Print out the title that was found
            print (f'Found  {title}')
        else:
            print(f"Failed to get movie ID, status code: {response.status_code}      {title}")
        
    except requests.exceptions.RequestException as e:
    # This catches all exceptions that are requests-related
    # Including connection errors, timeouts, etc.
        print("A network error occurred. Please try again later.")
        print(e)
display (tmbd_movies_list)


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print (json.dumps(tmbd_movies_list[:5],indent=4))

In [ ]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmbd_movies_list)
display (tmdb_movies_df.head(3))
display (love_movie_review_df.head(3))


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
tmdb_movies_df.rename(columns={'movie_title':'title'}, inplace=True)
merged_df = pd.merge(tmdb_movies_df, love_movie_review_df, on='title')
nyt_tmdb_df = merged_df.copy()
print (merged_df.columns)
print (merged_df.head(2))

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
def contains_list(series):
    return series.map(lambda x: isinstance(x, list)).any()

columns_with_lists = [col for col in nyt_tmdb_df.columns if contains_list(nyt_tmdb_df[col])]

print  (f'\n{columns_with_lists}\n' )
# Create a list of characters to remove
def clean_list_string(s):
    if isinstance(s, list):
        # Convert list to string and remove unwanted characters
        return str(s).strip('[]').replace('"', '').replace("'", "")
    return s

# Apply the cleaning directly, no need for preliminary list check
for col in columns_with_lists:
   nyt_tmdb_df[col] = nyt_tmdb_df[col].apply(clean_list_string)

# Display the fixed DataFrame
print (nyt_tmdb_df.head(2))
print (nyt_tmdb_df.columns)


In [ ]:
# Drop "byline.person" column
nyt_tmdb_df.drop(columns=['byline.person'], inplace=True)

In [ ]:
# Delete duplicate rows and reset index
print(nyt_tmdb_df[nyt_tmdb_df.duplicated(keep=False)])
#duplicates = nyt_tmdb_df.duplicated()
#print(duplicates)

In [ ]:
# Export data to CSV without the index
